# Actual experiment 

In [1]:
import numpy as np
from tqdm.notebook import tqdm
from time import sleep
import pylab
import os
import matplotlib.pyplot as plt
from matplotlib import style
#speed things up
import numba
from numba import njit
from numba import jit
from numba import prange
from numba_progress import ProgressBar
from scipy.ndimage import convolve, generate_binary_structure
from timeit import default_timer as timer
#style
plt.style.use(['science','notebook','grid'])

In [2]:
f_ext=0

In [3]:
#funzione che calcola che tiene conto di PBC per un generico reticolo rettangolare (nlatt_x,nlatt_y)
def boundaries_cond(nlatt_x,nlatt_y):
    nlatt_x = int(nlatt_x)
    nlatt_y = int(nlatt_y)
    if (nlatt_x == nlatt_y):
        npp = np.zeros(nlatt_x).astype(np.int64)
        nmm = np.zeros(nlatt_x).astype(np.int64)
        
        for i in range(nlatt_x):
            npp[i] = i+1
            nmm[i] = i-1
        npp[nlatt_x - 1] = 0
        nmm[0] = nlatt_x - 1
        return npp,nmm

    else:
        nppx = np.zeros(nlatt_x).astype(np.int64)
        nmmx = np.zeros(nlatt_x).astype(np.int64)
        nppy = np.zeros(nlatt_y).astype(np.int64)
        nmmy = np.zeros(nlatt_y).astype(np.int64)
        for i in range(nlatt_x):
            nppx[i] = i + 1
            nmmx[i] = i - 1
        nppx[nlatt_x - 1] = 0
        nmmx[0]= nlatt_x - 1

        for i in range(nlatt_y):
            nppy[i] = i+1    
            nmmy[i] = i-1
        nppy[nlatt_y - 1] = 0
        nmmy[0]  = nlatt_y - 1

        return nppx,nmmx,nppy,nmmy 

In [4]:
@jit(parallel=True)
def init(flag, Nx, Ny):   # 0 parte a freddo , 1 a caldo 50 su e 50 giu, ----> aggiungere carica da file o partenze particolari
    field = np.zeros((Nx,Ny))
    if (flag == 0):      
        field[:][:] = 1
        return field
    elif (flag == 1):
        for i in prange (0,Nx):
            for j in prange(0,Ny):
                r = np.random.uniform(0,1)
                if (r<0.5):
                     field[i][j] = 1
                else:
                    field[i][j] = -1
        return field 

In [5]:
@njit("(f8[:,:],f8, i8[:], i8[:])", cache=True)
def metropolis(field, beta, npp, nmm):
    nvol=int(field.shape[0]*field.shape[1])
    Nx = field.shape[0]
    Ny = field.shape[1]
    for i in range(0,nvol):
        x = int(np.random.uniform(0,1) * Nx)    #python indicizza da 0 a n-1 per a[n]
        y = int(np.random.uniform(0,1) * Ny)
        
        xp = npp[x]
        xm = nmm[x]
        yp = npp[y]
        ym = nmm[y]
        
        f = beta * (field[x][yp] + field[x][ym] + field[xp][y] + field[xm][y] + f_ext)   #calcolo la 'forza' + eventuale campo esterno
        
        s_i = field[x][y]    #spin attuale
        
        p_rat = np.exp(-2 * f * s_i)
        r = np.random.uniform(0,1)     #test accettanza
        
        if(r < p_rat):
            field[x][y] = -1*s_i

In [6]:
@njit("f8(f8[:,:], i8[:], i8[:])")
def energy(field, npp, nmm):
    ene=0.0
    for x in range(0,field.shape[0]):
        for y in range(0,field.shape[1]):
            xp = npp[x]
            xm = nmm[x]
            yp = npp[y]
            ym = nmm[y]
            f = field[x][yp] + field[x][ym] + field[xp][y] + field[xm][y]
            ene = ene -  0.5*f*field[x,y]
            ene = ene - f_ext*field[x,y]
    ene = ene/(field.shape[0]*field.shape[1])
    return ene  

In [7]:
@njit("f8(f8[:,:], i8[:], i8[:])")
def magnetization(field, npp, nmm):
    magn=0.0
    for x in prange(0, field.shape[0]):
        for y in prange(0, field.shape[1]):
            magn += field[x,y]
    magn = magn/(field.shape[0]*field.shape[1])
    return magn

In [8]:
beta_exp = np.round(np.arange(0.3,0.55,0.002), decimals=3)
beta_exp_try = np.round(np.arange(0.3,0.31,0.001), decimals=3)
dim = np.arange(10,75,10)
dim_try =np.arange(10,30,10)
#for simplicity let's consider only a square like lattice
N=50
save_measure=25
measures=2000000
measures_try=10000

In [ ]:
def sim(beta, measures):
    npp, nmm = boundaries_cond(N,N)
    file=open('simulation_test.txt',"w")
    file.write("#betas\t\tEnergy\t\tMagn\n")
    for bs in tqdm(beta):
        field = init(1, N, N)
        for meas in range(measures):
            metropolis(field, bs, npp, nmm)
            if(meas%save_measure==0):
                file.write("%f\t\t%f.8\t\t%.8f\n"%(bs, energy(field, npp, nmm), magnetization(field, npp, nmm)))
    file.close()

In [ ]:
sim(beta_exp,measures)

In [ ]:
beta, meas, es, mgns = np.loadtxt('a_simulation_30_old.txt', unpack=True)

In [ ]:
asmatrix = np.column_stack((beta, meas, mgns))
bs=asmatrix[:,0]
asmatrix[bs==0.3,2]

In [ ]:
m=[]
for bs in tqdm(beta_exp):
    indices=asmatrix[:,0]
    mgns= np.sum(np.abs(asmatrix[(indices == bs),2]))
    mgns = mgns/(len(asmatrix[(indices == bs),2]))
    m.append(mgns)
    


In [ ]:
plt.figure(figsize=(20,10))
plt.plot(beta_exp,m,'.')
plt.show()

In [9]:
def sim(beta, measures, save_step, n_latt):
    npp, nmm = boundaries_cond(n_latt,n_latt)
    filename=f"simulation_{n_latt}.txt"
    file = open(filename,"w")
    file.write("#betas\t\tEnergy\t\tMagn\n")
    for bs in beta:
        field = init(1, n_latt, n_latt)
        for meas in range(measures):
            metropolis(field, bs, npp, nmm)
            if(meas%save_measure==0):
                file.write("%f\t\t%f\t\t%f\n"%(bs, energy(field, npp, nmm), magnetization(field, npp, nmm)))
    file.close()

In [10]:
def sim_n(beta, measures, save_step, n_latt):
    for n in tqdm(n_latt):
        sim(beta, measures, save_step, n)

In [11]:
sim_n(beta_exp, measures, save_measure, dim)

  0%|          | 0/7 [00:00<?, ?it/s]

/var/folders/7b/mxhdlhl16yjddckjcf2qpms00000gn/T/ipykernel_15983/877078576.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "init" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../../../../../../var/folders/7b/mxhdlhl16yjddckjcf2qpms00000gn/T/ipykernel_15983/877078576.py", line 8:
<source missing, REPL/exec in use?>

  @jit(parallel=True)
/opt/homebrew/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "init" was compiled in object mode without forceobj=True, but has lifted loops.

File "../../../../../../var/folders/7b/mxhdlhl16yjddckjcf2qpms00000gn/T/ipykernel_15983/877078576.py", line 3:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/homebrew/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to th

In [ ]:
def prova():
    for i in range(10):
        filename=f"file_{i}"
        with open(filename,"a") as file:
            for j in range(1000):
                file.write("%i\t\n"%(i)) 

In [ ]:
%time prova()